In [1]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import sys
display(sys.version)

'3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]'

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

display(np.__version__)
display(pd.__version__)

'1.20.2'

'1.2.4'

# Import DESeq2 differential analysis of CEV-v1 CRISPRi screens

In [3]:
#sgRNA-level results for human, chimpanzee, and chimpanzee-human terms

human_deseq = pd.read_csv('/home/tdfair/Desktop/datatables/human_CEV-v1_DESeq2.csv')
chimp_deseq = pd.read_csv('/home/tdfair/Desktop/datatables/chimp_CEV-v1_DESeq2.csv')
chimp_minus_human_deseq = pd.read_csv('/home/tdfair/Desktop/datatables/chimp_minus_human_CEV-v1_DESeq2.csv')

In [4]:
pattern = r"(.*)__(.*)"

def extract_pattern(df):
    df = df['gene_TSS'].str.extract(pattern).rename(columns={0: 'gene', 1: 'TSS'}).join(df)
    return df

human_deseq, chimp_deseq, chimp_minus_human_deseq = [extract_pattern(df) for df in [human_deseq, chimp_deseq, chimp_minus_human_deseq]]

# Use α-RRA to combine sgRNA p-values into gene FDRs

α-RRA is called from MAGeCK (Li et al., *Genome Biol* 2014)

In [5]:
#α-RRA formatting

for df in [human_deseq, chimp_deseq, chimp_minus_human_deseq]:
    df['RRA'] = 1

In [6]:
#α-RRA input file format, sgRNAs rankings are based on adjusted p-values from DESeq2 

df = [human_deseq, chimp_deseq, chimp_minus_human_deseq]
file_name = ['human', 'chimp', 'chimp_minus_human']

for df,file in zip(df, file_name):
    df[['sgID', 'gene_TSS', 'RRA', 'padj', 'RRA', 'RRA']].to_csv(f'/home/tdfair/Desktop/datatables/{file}_CEV-v1_DESeq2_RRA_input.txt', sep='\t', index=False, header=False)

In [7]:
#identify negative-control sgRNAs for α-RRA

human_deseq.loc[human_deseq['gene'] == 'negative_control']['sgID'].to_csv('/home/tdfair/Desktop/datatables/CEV-v1_negative_control_sgRNAs.txt', sep='\n', index=False, header=False)

In [8]:
#Critical to adjust α in α-RRA to remove the effect of insignificant sgRNAs from the assessment of gene significance. 

#This step is analogous to the --gene-test-fdr-threshold argument in mageck test. 

#Two criteria for inclusion: 
    #sgRNA adjusted p-value < 0.01
    #sgRNA log2 fold-change <= -0.5 or >= 0.5
    
#This threshold (α) is the fraction of sgRNAs (ranked by adjusted p-values) retained for analysis by α-RRA.

for df in [human_deseq, chimp_deseq, chimp_minus_human_deseq]:
    print('%.20f' % (df.loc[(df['padj'] < 0.01) & (abs(df['log2FoldChange']) >= 0.50)].shape[0] / df.shape[0]))

0.30091875709714049902
0.23691545370083616495
0.11696087540002064742


In [9]:
cd /home/tdfair/Desktop/datatables

/home/tdfair/Desktop/datatables


In [10]:
!RRA \
-i human_CEV-v1_DESeq2_RRA_input.txt \
--control CEV-v1_negative_control_sgRNAs.txt \
-o human_CEV-v1_DESeq2_RRA_output.txt \
--permutation 1000 \
-p 0.30091875709714049902

Welcome to RRA v 0.5.9.
1845 control sequences loaded.
Reading input file...
Summary: 9686 sgRNAs, 972 genes, 1 lists; skipped sgRNAs:0
Computing lo-values for each group...
Skipping gene negative_control__na for permutation ...
Computing false discovery rate...
Total # control sgRNAs: 1844
Permuting genes with 4 sgRNAs...
Permuting genes with 7 sgRNAs...
Permuting genes with 8 sgRNAs...
Permuting genes with 9 sgRNAs...
Permuting genes with 11 sgRNAs...
Number of genes under FDR adjustment: 972
Saving to output file...
Suppressing the output of gene negative_control__na since it is negative ontrol genes.
RRA completed.


In [11]:
!RRA \
-i chimp_CEV-v1_DESeq2_RRA_input.txt \
--control CEV-v1_negative_control_sgRNAs.txt \
-o chimp_CEV-v1_DESeq2_RRA_output.txt \
--permutation 1000 \
-p 0.23691545370083616495

Welcome to RRA v 0.5.9.
1845 control sequences loaded.
Reading input file...
Summary: 9686 sgRNAs, 972 genes, 1 lists; skipped sgRNAs:0
Computing lo-values for each group...
Skipping gene negative_control__na for permutation ...
Computing false discovery rate...
Total # control sgRNAs: 1844
Permuting genes with 4 sgRNAs...
Permuting genes with 7 sgRNAs...
Permuting genes with 8 sgRNAs...
Permuting genes with 9 sgRNAs...
Permuting genes with 11 sgRNAs...
Number of genes under FDR adjustment: 972
Saving to output file...
Suppressing the output of gene negative_control__na since it is negative ontrol genes.
RRA completed.


In [12]:
!RRA \
-i chimp_minus_human_CEV-v1_DESeq2_RRA_input.txt \
--control CEV-v1_negative_control_sgRNAs.txt \
-o chimp_minus_human_CEV-v1_DESeq2_RRA_output.txt \
--permutation 1000 \
-p 0.11696087540002064742

Welcome to RRA v 0.5.9.
1845 control sequences loaded.
Reading input file...
Summary: 9686 sgRNAs, 972 genes, 1 lists; skipped sgRNAs:0
Computing lo-values for each group...
Skipping gene negative_control__na for permutation ...
Computing false discovery rate...
Total # control sgRNAs: 1844
Permuting genes with 4 sgRNAs...
Permuting genes with 7 sgRNAs...
Permuting genes with 8 sgRNAs...
Permuting genes with 9 sgRNAs...
Permuting genes with 11 sgRNAs...
Number of genes under FDR adjustment: 972
Saving to output file...
Suppressing the output of gene negative_control__na since it is negative ontrol genes.
RRA completed.


# Import α-RRA analysis

In [13]:
chimp_rra_deseq = pd.read_csv('chimp_CEV-v1_DESeq2_RRA_output.txt', sep='\t')
human_rra_deseq = pd.read_csv('human_CEV-v1_DESeq2_RRA_output.txt', sep='\t')
chimp_minus_human_rra_deseq = pd.read_csv('chimp_minus_human_CEV-v1_DESeq2_RRA_output.txt', sep='\t')

In [14]:
pattern = r"(.*)__(.*)"

def extract_pattern(df):
    df = df['group_id'].str.extract(pattern).rename(columns={0: 'gene', 1: 'TSS'}).join(df)
    return df

human_rra_deseq, chimp_rra_deseq, chimp_minus_human_rra_deseq = [extract_pattern(df) for df in [human_rra_deseq, chimp_rra_deseq, chimp_minus_human_rra_deseq]]

# Identify genes with species-specific effects

In [15]:
def shared_list(list_1, list_2):
    return list(set(list_1) & set(list_2))

In [16]:
#calculate gene-level phenotypes, use top N sgRNAs as ranked by absolute log2 fold-change

N = 4

human_rra_deseq = human_rra_deseq.merge(human_deseq.sort_values('log2FoldChange', key=pd.Series.abs, ascending=False).groupby('gene_TSS').head(N).groupby('gene_TSS')['log2FoldChange'].mean(), right_index=True, left_on='group_id', how='left')
chimp_rra_deseq = chimp_rra_deseq.merge(chimp_deseq.sort_values('log2FoldChange', key=pd.Series.abs, ascending=False).groupby('gene_TSS').head(N).groupby('gene_TSS')['log2FoldChange'].mean(), right_index=True, left_on='group_id', how='left')
chimp_minus_human_rra_deseq = chimp_minus_human_rra_deseq.merge(chimp_minus_human_deseq.sort_values('log2FoldChange', key=pd.Series.abs, ascending=False).groupby('gene_TSS').head(N).groupby('gene_TSS')['log2FoldChange'].mean(), right_index=True, left_on='group_id', how='left')

In [17]:
def positive_log2FC(df):
    if df['log2FoldChange'] > 0:
        return True
    return False

In [18]:
#label genes with positive fold-change

human_rra_deseq['positive_log2FC_human'] = human_rra_deseq.apply(positive_log2FC, axis=1)
chimp_rra_deseq['positive_log2FC_chimpanzee'] = chimp_rra_deseq.apply(positive_log2FC, axis=1)

In [19]:
#Must exclude genes with shared effects from being erroneously called as species-specific

#One approach: discard any gene with a FDR in both the human and chimpanzee species terms less than the highest FDR for any gene with at least one sgRNA passing α threshold

h_max = human_rra_deseq.loc[human_rra_deseq['goodsgrna'] >= 1]['FDR'].max()
c_max = chimp_rra_deseq.loc[chimp_rra_deseq['goodsgrna'] >= 1]['FDR'].max()

a = human_rra_deseq.loc[human_rra_deseq['FDR'] <= h_max]['group_id']
b = chimp_rra_deseq.loc[chimp_rra_deseq['FDR'] <= c_max]['group_id']

both_species_hits = shared_list(a,b)
len(both_species_hits)

546

In [20]:
#Merge genes with possible shared effects to identify genes with opposite direction phenotypes (enriched & essential)

a = human_rra_deseq.loc[human_rra_deseq['group_id'].isin(both_species_hits)].sort_values('group_id')[['group_id', 'positive_log2FC_human']]
b = chimp_rra_deseq.loc[chimp_rra_deseq['group_id'].isin(both_species_hits)].sort_values('group_id')[['group_id', 'positive_log2FC_chimpanzee']]

combined_hits = a.merge(b)

In [21]:
#Retain genes with opposite direction phenotypes (enriched & essential) between species

opposites = combined_hits.loc[(combined_hits['positive_log2FC_human'] != combined_hits['positive_log2FC_chimpanzee'])]
opposites

,group_id,positive_log2FC_human,positive_log2FC_chimpanzee
8,AES__P1P2,False,True
11,AJUBA__P1P2,False,True
23,ARAP3__P1P2,False,True
26,ARHGAP21__P1P2,False,True
38,BCL2L13__P1P2,True,False
44,BRPF1__P1P2,False,True
49,C19orf43__P1P2,False,True
61,CBLL1__P1P2,False,True
67,CCNE1__P1P2,True,False
83,CLK3__P2,True,False


In [22]:
#Exclude genes with shared effects, unless the effect is of opposite direction phenotypes (enriched & essential) between species:

chimp_minus_human_no_shared_deseq = chimp_minus_human_rra_deseq.loc[(~chimp_minus_human_rra_deseq['group_id'].isin(both_species_hits)) | (chimp_minus_human_rra_deseq['group_id'].isin(opposites['group_id']))]
chimp_minus_human_no_shared_deseq

,gene,TSS,group_id,items_in_group,lo_value,p,FDR,goodsgrna,log2FoldChange
0,PAN2,P1P2,PAN2__P1P2,8,4.721500e-17,5.138900e-07,0.000009,7,-2.614755
1,ATP6AP1,P1P2,ATP6AP1__P1P2,8,9.493100e-15,5.138900e-07,0.000009,8,3.179830
2,NKRF,P1P2,NKRF__P1P2,8,5.001800e-11,5.138900e-07,0.000009,8,-2.729973
3,CCNE1,P1P2,CCNE1__P1P2,8,1.700400e-10,5.138900e-07,0.000009,8,-1.428967
4,CDK2,P1P2,CDK2__P1P2,8,3.543300e-10,5.138900e-07,0.000009,8,-2.030713
...,...,...,...,...,...,...,...,...,...
958,FNIP1,P1P2,FNIP1__P1P2,8,9.969400e-01,9.554000e-01,0.967344,0,0.104631
961,WNT3A,P1P2,WNT3A__P1P2,8,9.976700e-01,9.645800e-01,0.973649,0,-0.056883
962,PTPN20B,48826780,PTPN20B__48826780,8,9.977700e-01,9.654000e-01,0.973649,0,-0.053514
963,ZNF644,P1P2,ZNF644__P1P2,8,9.977800e-01,9.656400e-01,0.973649,0,-0.144575


In [23]:
chimp_minus_human_no_shared_deseq.loc[chimp_minus_human_no_shared_deseq['FDR'] < 0.01]

,gene,TSS,group_id,items_in_group,lo_value,p,FDR,goodsgrna,log2FoldChange
0,PAN2,P1P2,PAN2__P1P2,8,4.721500e-17,5.138900e-07,0.000009,7,-2.614755
1,ATP6AP1,P1P2,ATP6AP1__P1P2,8,9.493100e-15,5.138900e-07,0.000009,8,3.179830
2,NKRF,P1P2,NKRF__P1P2,8,5.001800e-11,5.138900e-07,0.000009,8,-2.729973
3,CCNE1,P1P2,CCNE1__P1P2,8,1.700400e-10,5.138900e-07,0.000009,8,-1.428967
4,CDK2,P1P2,CDK2__P1P2,8,3.543300e-10,5.138900e-07,0.000009,8,-2.030713
...,...,...,...,...,...,...,...,...,...
123,TMED2,P1P2,TMED2__P1P2,8,1.609500e-02,1.289900e-04,0.000951,3,-1.052522
125,PLOD3,P1P2,PLOD3__P1P2,8,1.798300e-02,1.289900e-04,0.000951,2,0.979755
128,CABIN1,P1P2,CABIN1__P1P2,8,2.241300e-02,1.300100e-04,0.000951,3,1.148833
129,STX4,P1P2,STX4__P1P2,8,2.409200e-02,1.300100e-04,0.000951,2,0.600473


In [24]:
#Set of 75 species-specific genetic dependencies (Fig. 2)

#Three criteria for inclusion: 
    #gene FDR < 0.01
    #at least three sgRNAs targeting the gene pass α threshold
    #gene log2 fold-change >= 0.75 between species

chimp_minus_human_no_shared_deseq.loc[(chimp_minus_human_no_shared_deseq['FDR'] < 0.01) & 
                                      (chimp_minus_human_no_shared_deseq['goodsgrna'] >= 3) & 
                                      (abs(chimp_minus_human_no_shared_deseq['log2FoldChange']) >= 0.75)]

,gene,TSS,group_id,items_in_group,lo_value,p,FDR,goodsgrna,log2FoldChange
0,PAN2,P1P2,PAN2__P1P2,8,4.721500e-17,5.138900e-07,0.000009,7,-2.614755
1,ATP6AP1,P1P2,ATP6AP1__P1P2,8,9.493100e-15,5.138900e-07,0.000009,8,3.179830
2,NKRF,P1P2,NKRF__P1P2,8,5.001800e-11,5.138900e-07,0.000009,8,-2.729973
3,CCNE1,P1P2,CCNE1__P1P2,8,1.700400e-10,5.138900e-07,0.000009,8,-1.428967
4,CDK2,P1P2,CDK2__P1P2,8,3.543300e-10,5.138900e-07,0.000009,8,-2.030713
...,...,...,...,...,...,...,...,...,...
114,MED23,P1P2,MED23__P1P2,8,8.306300e-03,8.479200e-05,0.000693,4,1.355786
120,MTCH2,P1P2,MTCH2__P1P2,8,1.212100e-02,1.279600e-04,0.000951,3,1.164425
121,MAZ,P1P2,MAZ__P1P2,8,1.379800e-02,1.279600e-04,0.000951,3,-0.950570
123,TMED2,P1P2,TMED2__P1P2,8,1.609500e-02,1.289900e-04,0.000951,3,-1.052522
